In [34]:
from pathlib import Path
import re
import numpy as np
import matplotlib.pyplot as plt
import spikeinterface.extractors as se
import RCP_analysis as rcp

# ------------ USER INPUTS ------------
FILE = Path('/home/bryan/mnt/cullen/Current Project Databases - NHP/2025 Cerebellum prosthesis/Nike/Utah_test002/Test_reaches001.ns6')
START, END = 20000, 25000      # sample window to preview
BR_IDX = 1                     # use the relevant BR block index if your mapping needs it (any int is OK if unused)
REPO_ROOT = Path.cwd()         # change if needed
PARAMS = rcp.load_experiment_params(REPO_ROOT.parents[0] / "config" / "params.yaml", repo_root=REPO_ROOT)
XLS = rcp.ua_excel_path(REPO_ROOT.parents[0], getattr(PARAMS, "probes", {}))
UA_MAP = rcp.load_UA_mapping_from_excel(XLS) if XLS and XLS.exists() else None
if UA_MAP is None:
    raise SystemExit("[error] UA mapping not found. Check XLS path or provide UA_MAP manually.")
DATA_ROOT = rcp.resolve_data_root(PARAMS)
metadata_rel = getattr(PARAMS, "metadata_rel", "")
METADATA_CSV = (DATA_ROOT / metadata_rel) if metadata_rel else None

# ----- impedance file(s) (hardcoded path you gave) -----
IMP_A = Path('/home/bryan/mnt/cullen/Current Project Databases - NHP/2025 Cerebellum prosthesis/Nike/Utah_test002/Port_A_impedence')
# If you also have Port B, add it here:
# IMP_B = Path('/home/bryan/.../Utah_test002/Port_B_impedence')

# ---------------- impedance helpers ----------------
def _read_text_loose(p: Path) -> str:
    b = p.read_bytes()
    for enc in ("utf-8", "utf-8-sig", "cp1252", "latin-1", "mac_roman"):
        try:
            return b.decode(enc)
        except Exception:
            pass
    filtered = bytes(ch for ch in b if 9 <= ch <= 126 or ch in (10, 13))
    return filtered.decode("latin-1", errors="ignore")

# Primary pattern like: "elec1-5   201 kOhm"  → captures (5, 201, kOhm)
_imp_pat_primary = re.compile(
    r"\belec\s*\d+\s*-\s*(\d{1,3})\s+([0-9]+(?:\.[0-9]+)?)\s*(k?ohms?|kΩ|ohms?|Ω)\b",
    flags=re.IGNORECASE,
)

# Fallback (looser): e.g., "5  201 kOhm", "elec 5 : 201 kΩ"
_imp_pat_fallback = re.compile(
    r"\b(?:elec(?:trode)?\s*)?(\d{1,3})\D{0,10}?([0-9]+(?:\.[0-9]+)?)\s*(k?ohms?|kΩ|ohms?|Ω)\b",
    flags=re.IGNORECASE,
)

def _unit_to_kohm(val: float, unit: str) -> float:
    return float(val) if "k" in (unit or "").lower() else float(val) / 1000.0

def load_impedances_from_text(path_like: Path) -> dict[int, float]:
    txt = _read_text_loose(path_like)
    out = {}
    matches = list(_imp_pat_primary.finditer(txt))
    if not matches:
        matches = list(_imp_pat_fallback.finditer(txt))
    for m in matches:
        elec_str, val_str, unit = m.groups()
        try:
            e = int(elec_str); v = float(val_str)
        except Exception:
            continue
        out[e] = _unit_to_kohm(v, unit)
    return out

def _fmt_impedance_kohm(z: float | None) -> str:
    if z is None or not np.isfinite(z): return "—"
    if z >= 1000: return f"{z:.0f} kΩ (open)"
    return f"{z:.0f} kΩ" if z >= 100 else f"{z:.1f} kΩ"
def _imp_color(z: float | None) -> str:
    """
    Three colors:
      - >= 1000 kΩ  → 'red'
      - 500–<1000 kΩ → 'tab:orange'
      - < 500 kΩ    → 'tab:blue'
      - unknown/NaN → 'gray'
    """
    if z is None or not np.isfinite(z):
        return "gray"
    if z >= 1000:
        return "red"
    if z >= 500:
        return "tab:orange"
    return "tab:blue"

# ---------------------------------------------------

def _parse_elec_from_id(cid):
    """
    Accepts e.g. 'UAe005_NSP001', 'UAe37', or '37' → returns an int electrode if possible.
    """
    s = str(cid)
    m = re.search(r"UAe(\d{1,3})", s, flags=re.IGNORECASE)
    if m:
        return int(m.group(1))
    m2 = re.search(r"(\d{1,3})$", s)
    return int(m2.group(1)) if m2 else None

# 1) Load Blackrock
rec_raw = se.read_blackrock(FILE)

# 2) Apply UA mapping
mapped_nsp = UA_MAP.get("mapped_nsp")
if mapped_nsp is None:
    raise SystemExit("[error] UA_MAP missing key 'mapped_nsp'")
rec_mapped, _ = rcp.apply_ua_mapping_by_renaming(rec_raw, mapped_nsp, BR_IDX, METADATA_CSV)

# 3) Sort channels by UA electrode number so 1..6 are true UA order
cids = list(rec_mapped.get_channel_ids())
elec_idx = []
unparsed_idx = []
for i, cid in enumerate(cids):
    e = _parse_elec_from_id(cid)
    (elec_idx if e is not None else unparsed_idx).append((i, e))
elec_idx.sort(key=lambda t: t[1] if t[1] is not None else 10**9)
sorted_indices = [i for (i, _) in elec_idx] + [i for (i, _) in unparsed_idx]

def _chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

# --- Load impedances from Port A (and optionally Port B) ---
imp_by_elec = {}
if IMP_A.exists():
    imp_by_elec.update(load_impedances_from_text(IMP_A))
    print(f"[imp] Loaded {len(imp_by_elec)} entries from {IMP_A}")
else:
    print(f"[imp] WARNING: impedance file not found: {IMP_A}")
# If you also have Port B, uncomment:
# if IMP_B.exists():
#     dB = load_impedances_from_text(IMP_B)
#     imp_by_elec.update(dB)
#     print(f"[imp] Loaded {len(dB)} entries from {IMP_B}")

# Sanity debug
print("[debug] first 8 channel IDs:", cids[:8])
print("[debug] first 8 parsed elec #:", [_parse_elec_from_id(s) for s in cids[:8]])

# 4) Fetch & plot ALL channels in groups of 6 (with impedance in ylabel / color-coded)
fs = rec_mapped.get_sampling_frequency()
n_samp = END - START
t_ms = np.arange(n_samp) / fs * 1000.0

for fig_idx, group in enumerate(_chunks(sorted_indices, 6), start=1):
    sel_ids = [cids[i] for i in group]
    tr = rec_mapped.get_traces(start_frame=START, end_frame=END, channel_ids=sel_ids)  # (n_samples, n_group)

    # Make a wide figure with up to 6 rows
    n_rows = len(sel_ids)
    fig, axes = plt.subplots(n_rows, 1, figsize=(16, 2.6*n_rows), sharex=True)
    if n_rows == 1:
        axes = [axes]

    for i, ax in enumerate(axes):
        cid = sel_ids[i]
        elec = _parse_elec_from_id(cid)
        z_kohm = imp_by_elec.get(elec) if elec is not None else None
        col = _imp_color(z_kohm)

        ax.plot(t_ms, tr[:, i], color=col, lw=1.05)
        ax.grid(True, alpha=0.3)
        ax.set_ylabel(f"{cid}\n({_fmt_impedance_kohm(z_kohm)})",
                      rotation=0, labelpad=25, va='center', color=col)
        ax.yaxis.set_label_coords(-0.10, 0.5)

    axes[-1].set_xlabel("Time (ms)")
    legend_key = "imp: ≥1000 kΩ = red • 500–<1000 kΩ = orange • <500 kΩ = blue"
    fig.suptitle(
        f"Samples {START}:{END} "
        f"({n_samp/fs:.3f}s) • group {fig_idx}\n{legend_key}",
        y=0.995
    )
    plt.tight_layout(rect=[0, 0.03, 1, 0.97])
    fig.savefig(f"ua_quickview_group{fig_idx:02d}.png", dpi=150); plt.close(fig) #plt.show()   # or: 


[MAP] renamed 128/128 rows with UA mapping ('UAe###_NSP###').
[imp] Loaded 128 entries from /home/bryan/mnt/cullen/Current Project Databases - NHP/2025 Cerebellum prosthesis/Nike/Utah_test002/Port_A_impedence
[debug] first 8 channel IDs: [np.str_('UAe005_NSP001'), np.str_('UAe014_NSP002'), np.str_('UAe025_NSP003'), np.str_('UAe038_NSP004'), np.str_('UAe053_NSP005'), np.str_('UAe070_NSP006'), np.str_('UAe006_NSP007'), np.str_('UAe015_NSP008')]
[debug] first 8 parsed elec #: [5, 14, 25, 38, 53, 70, 6, 15]
